In [87]:
from queries import *
from pathlib import Path
from pandas_profiling import ProfileReport
from sql_metadata import Parser
import sqlparse
import sqlglot
import mo_sql_parsing as mo
import re
from pprint import pprint

In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

In [3]:
engine = connect_sqlserver_windowsauth('bidatacentre','master')
db_df = fetch_all_dbs(engine)


In [4]:
def markdown_table(in_df):
    header = ""
    line2 = ""
    data = ""
    for col in in_df.columns:
        header = header + f"| {col} "
        line2 = line2 + f"| --- "
    header = header + "|\n"
    line2 = line2 + "|\n"
    for row in in_df.itertuples():
        for col in range(1,len(row)):
            data = data + f"| {row[col]} "
        data = data + "|\n"
    table_string = header + line2 + data + "\n"
    return table_string

In [5]:
# Test code
deps = fetch_all_db_dependencies(engine)

In [6]:
deps


,DATABASE,referencing_schema_name,referencing_object_name,referencing_type_desc,referenced_server_name,referenced_db_name,referenced_schema_name,referenced_entity_name
0,APPS,dbo,AX_GL_BudgetDetails,VIEW,None,BI,dbo,AX_Budget_Trans
1,APPS,dbo,AX_GL_Master,VIEW,None,AX_SECURE_PRD_REPLICA,dbo,MAINACCOUNT
2,APPS,dbo,AX_GL_Master,VIEW,None,AX_SECURE_PRD_REPLICA,dbo,MAINACCOUNTCATEGORY
3,APPS,dbo,AX_GL_TransDetails,VIEW,None,BI,dbo,AX_GL_TransDetails
4,APPS,dbo,AX_PChemCustomer,VIEW,None,BI,dbo,AX_Customer_Master
...,...,...,...,...,...,...,...,...
41519,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectByWorkflowSubreportRS,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,TaskStatusTranslated
41520,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectByWorkflowSubreportRS,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,TaskTypesTranslated
41521,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectByWorkflowSubreportRS,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,WorkflowTasks
41522,iTrak3_SecureEnergy,dbo,WorkflowTasksSelectList,SQL_STORED_PROCEDURE,None,iTrak3_SecureEnergy,dbo,WorkflowTasks


In [7]:
get_children(deps, 'BI', 'dbo', 'AX_Budget_Trans')

,DATABASE,referencing_schema_name,referencing_object_name,referencing_type_desc
0,APPS,dbo,AX_GL_BudgetDetails,VIEW
24974,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-01 Facility Roll Up Print Cop...,SQL_STORED_PROCEDURE
24975,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-01 Facility Roll Up Print Cop...,SQL_STORED_PROCEDURE
24979,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-01 Facility Roll Up Print Cop...,SQL_STORED_PROCEDURE
24980,AX_SECURE_PRD_REPLICA,temp,/_Master/PRD-FIN-02 PRD Summary Report - CAN__1,SQL_STORED_PROCEDURE
1217,BI,dbo,zPowerBI_PRDQForecast,VIEW
1455,BI,temp,/Report Parts/Admin Temp/PRD-FIN-XX PRD Summar...,SQL_STORED_PROCEDURE


## VIEWS

In [ ]:
engine = connect_sqlserver_windowsauth('bidatacentre','BI')
vcd = fetch_view_column_details(engine)
vcd.nunique()

In [33]:
# Most common columns in views
vcd.column_name.value_counts()

Company                 79
Location                50
Division                46
BusinessUnit            42
ProjectID               40
                        ..
BusinessDimension        1
BillableExportID         1
Billable_ExportMonth     1
Billable_ExportDate      1
Sub_Expiry               1
Name: column_name, Length: 2500, dtype: int64

In [24]:
view_def = fetch_view_definitions(engine)
view_def.nunique()

schema_name        2
view_name        214
created          214
last_modified    214
definition       214
comments           0
dtype: int64

In [102]:
# Get 1 definition to play with
defn = view_def['definition'][99]
print(defn)












CREATE VIEW [dbo].[FC_Truck_Ticket_View] AS

SELECT * FROM OPENQUERY(FCP, '

SELECT T1.TICKET_INDEX,
	   T2.TICKET_ID,
	   T1.DEVICE_NUMBER,
	   T2.HOST_ID,
	   T8.COMPANY_NAME AS CUSTOMER,
	   T3.NAME AS PRODUCT_NAME,
	   T5.DESTINATION_ORIGIN_LABEL AS ORIGIN,
	   T4.DESTINATION_ORIGIN_LABEL AS DESTINATION,
	   T7.COMPANY_NAME AS TRANSPORTER,
	   T6.COMPANY_NAME AS SHIPPER,
	   T9.COMPANY_NAME AS COSIGNOR,
	   T10.COMPANY_NAME AS CONSIGNEE,
	   T1.SEQUENCE_NUMBER,
	   T1.EFFECTIVE_DATE,
	   T1.GMS_DATE,
	   T1.GMS_USER,
	   T1.DEVICE_TYPE,
	   T1.ORIGIN_INDEX,
	   T1.DESTINATION_INDEX,
	   T1.SHIPPER_INDEX,
	   T1.TRANSPORTER_INDEX,
	   T1.PRODUCT_INDEX,
	   T1.MANIFEST_ID,
	   T1.MOVEMENT_START_DATE,
	   T1.MOVEMENT_END_DATE,
	   T1.PRESSURE,
	   T1.PULSES,
	   T1.INDICATED_VOLUME,
	   T1.GROSS_VOLUME,
	   T1.GROSS_STD_VOLUME,
	   T1.NET_STD_VOLUME,
	   T1.SEDIMENT_WATER_VOLUME,
	   T1.SEDIMENT_WATER_CORRECTION,
	   T1.OBSERVED_DENSITY,
	   T1.DENSITOMETER_RESULTS,
	   T1.

In [111]:
#  defn.replace("CREATE VIEW _anything_ AS","")
new_defn = re.sub("CREATE VIEW (.*) AS", "", defn)
# Remove [ ]  from query text
new_defn = new_defn.replace("[","").replace("]","").replace("\r","").replace("\n","").replace("\t","    ")
print(new_defn)

SELECT * FROM OPENQUERY(FCP, 'SELECT T1.TICKET_INDEX,       T2.TICKET_ID,       T1.DEVICE_NUMBER,       T2.HOST_ID,       T8.COMPANY_NAME AS CUSTOMER,       T3.NAME AS PRODUCT_NAME,       T5.DESTINATION_ORIGIN_LABEL AS ORIGIN,       T4.DESTINATION_ORIGIN_LABEL AS DESTINATION,       T7.COMPANY_NAME AS TRANSPORTER,       T6.COMPANY_NAME AS SHIPPER,       T9.COMPANY_NAME AS COSIGNOR,       T10.COMPANY_NAME AS CONSIGNEE,       T1.SEQUENCE_NUMBER,       T1.EFFECTIVE_DATE,       T1.GMS_DATE,       T1.GMS_USER,       T1.DEVICE_TYPE,       T1.ORIGIN_INDEX,       T1.DESTINATION_INDEX,       T1.SHIPPER_INDEX,       T1.TRANSPORTER_INDEX,       T1.PRODUCT_INDEX,       T1.MANIFEST_ID,       T1.MOVEMENT_START_DATE,       T1.MOVEMENT_END_DATE,       T1.PRESSURE,       T1.PULSES,       T1.INDICATED_VOLUME,       T1.GROSS_VOLUME,       T1.GROSS_STD_VOLUME,       T1.NET_STD_VOLUME,       T1.SEDIMENT_WATER_VOLUME,       T1.SEDIMENT_WATER_CORRECTION,       T1.OBSERVED_DENSITY,       T1.DENSITOMETER_RESULT

### Cleanup SQL formatting for display to user

In [112]:
reformatted = sqlparse.format(new_defn, reindent=True, keyword_case='upper')
reformatted

"SELECT *\nFROM OPENQUERY(FCP, 'SELECT T1.TICKET_INDEX,       T2.TICKET_ID,       T1.DEVICE_NUMBER,       T2.HOST_ID,       T8.COMPANY_NAME AS CUSTOMER,       T3.NAME AS PRODUCT_NAME,       T5.DESTINATION_ORIGIN_LABEL AS ORIGIN,       T4.DESTINATION_ORIGIN_LABEL AS DESTINATION,       T7.COMPANY_NAME AS TRANSPORTER,       T6.COMPANY_NAME AS SHIPPER,       T9.COMPANY_NAME AS COSIGNOR,       T10.COMPANY_NAME AS CONSIGNEE,       T1.SEQUENCE_NUMBER,       T1.EFFECTIVE_DATE,       T1.GMS_DATE,       T1.GMS_USER,       T1.DEVICE_TYPE,       T1.ORIGIN_INDEX,       T1.DESTINATION_INDEX,       T1.SHIPPER_INDEX,       T1.TRANSPORTER_INDEX,       T1.PRODUCT_INDEX,       T1.MANIFEST_ID,       T1.MOVEMENT_START_DATE,       T1.MOVEMENT_END_DATE,       T1.PRESSURE,       T1.PULSES,       T1.INDICATED_VOLUME,       T1.GROSS_VOLUME,       T1.GROSS_STD_VOLUME,       T1.NET_STD_VOLUME,       T1.SEDIMENT_WATER_VOLUME,       T1.SEDIMENT_WATER_CORRECTION,       T1.OBSERVED_DENSITY,       T1.DENSITOMETER_RESU

### Testing with mo_sql_parse

In [113]:
parsed_dict =  mo.parse_sqlserver(reformatted)
pprint(parsed_dict)

{'from': {'openquery': ['FCP',
                        {'literal': 'SELECT T1.TICKET_INDEX,       '
                                    'T2.TICKET_ID,       '
                                    'T1.DEVICE_NUMBER,       T2.HOST_ID,       '
                                    'T8.COMPANY_NAME AS CUSTOMER,       '
                                    'T3.NAME AS PRODUCT_NAME,       '
                                    'T5.DESTINATION_ORIGIN_LABEL AS '
                                    'ORIGIN,       T4.DESTINATION_ORIGIN_LABEL '
                                    'AS DESTINATION,       T7.COMPANY_NAME AS '
                                    'TRANSPORTER,       T6.COMPANY_NAME AS '
                                    'SHIPPER,       T9.COMPANY_NAME AS '
                                    'COSIGNOR,       T10.COMPANY_NAME AS '
                                    'CONSIGNEE,       '
                                    'T1.SEQUENCE_NUMBER,       '
                                    'T1

# Testing with sqlglot

### Match table to table alias

In [60]:
t_list=[]
for table in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.Table):
    t_list.append(table)
ta_list=[]

for tablealias in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.TableAlias):
    ta_list.append(tablealias)

print(len(t_list), len(ta_list))

for t,ta in zip(t_list, ta_list):
    print(f"{t} ---> {ta}")

4 3
AX_SECURE_PRD_REPLICA.dbo.docutype ---> DF
AX_SECURE_PRD_REPLICA.dbo.DOCUREF ---> t0
AX_SECURE_PRD_REPLICA.dbo.ledgerjournaltable ---> DV


In [62]:
for column in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.Column):
    print(column)

t0.DATAAREAID
t0.POSTEDDATETIME
t0.JOURNALNUM
df.NAME
dv.FILETYPE
docutype.ARCHIVEPATH
t0.DATAAREAID
t0.POSTEDDATETIME
docutype.archivepath
dv.filename
dv.filetype
t0.journalnum
df.NAME
dv.FILETYPE
t0.dataareaid
t0.journalnum
DF.typeid
docutype.typeid
DF.valuerecid
dv.recid
df.reftableid
dv.FileName
DF.refrecid
t0.RECID
t0.DATAAREAID
docutype.DATAAREAID
dv.filetype
t0.dataareaid
t0.journalnum
dv.filetype
docutype.ARCHIVEPATH
dv.fileNAME
docutype.ARCHIVEPATH
dv.fileNAME
dv.filetype
docutype.ARCHIVEPATH
dv.fileNAME


In [64]:
for select in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.Select):
  for projection in select.args["expressions"]:
    print(projection.alias_or_name)

Company
PostedDate
JournalBatchNumber
FileDescription
FileName
FileType
FilePath
WebPath
AttachmentNum


In [76]:
for i in select.args["expressions"]:
    print( i )
    print(repr(i))
    print("\n")

t0.DATAAREAID AS Company
(ALIAS this: 
  (COLUMN this: 
    (IDENTIFIER this: DATAAREAID, quoted: False), table: 
    (IDENTIFIER this: t0, quoted: False)), alias: 
  (IDENTIFIER this: Company, quoted: False))


t0.POSTEDDATETIME AS PostedDate
(ALIAS this: 
  (COLUMN this: 
    (IDENTIFIER this: POSTEDDATETIME, quoted: False), table: 
    (IDENTIFIER this: t0, quoted: False)), alias: 
  (IDENTIFIER this: PostedDate, quoted: False))


t0.JOURNALNUM AS JournalBatchNumber
(ALIAS this: 
  (COLUMN this: 
    (IDENTIFIER this: JOURNALNUM, quoted: False), table: 
    (IDENTIFIER this: t0, quoted: False)), alias: 
  (IDENTIFIER this: JournalBatchNumber, quoted: False))


df.NAME AS FileDescription
(ALIAS this: 
  (COLUMN this: 
    (IDENTIFIER this: NAME, quoted: False), table: 
    (IDENTIFIER this: df, quoted: False)), alias: 
  (IDENTIFIER this: FileDescription, quoted: False))


dv.FileName + '.' + dv.FILETYPE AS FileName
(ALIAS this: 
  (ADD this: 
    (ADD this: 
      (COLUMN this: 
   

In [78]:
type( select.args["expressions"][0] )

sqlglot.expressions.Alias

In [80]:
for i in sqlglot.parse_one(reformatted).find_all(sqlglot.exp.Alias):
    print(i)

t0.DATAAREAID AS Company
t0.POSTEDDATETIME AS PostedDate
t0.JOURNALNUM AS JournalBatchNumber
df.NAME AS FileDescription
dv.FileName + '.' + dv.FILETYPE AS FileName
dv.FILETYPE AS FileType
docutype.ARCHIVEPATH AS FilePath
REPLACE(REPLACE(docutype.ARCHIVEPATH + dv.fileNAME + '.' + dv.filetype, '\\', '/////'), '\', '/') AS WebPath
ROW_NUMBER() OVER(PARTITION BY t0.dataareaid, t0.journalnum ORDER BY t0.dataareaid, t0.journalnum, (docutype.ARCHIVEPATH + dv.fileNAME + dv.filetype)) AS AttachmentNum
AX_SECURE_PRD_REPLICA.dbo.DOCUREF AS DF
AX_SECURE_PRD_REPLICA.dbo.ledgerjournaltable AS t0
AX_SECURE_PRD_REPLICA.dbo.docuvalue AS DV


# WORKING

In [116]:
engine = connect_sqlserver_windowsauth('bidatacentre','master')
db_df = fetch_all_dbs(engine)

#for db in list(db_df['name']):
db = 'BI'
engine = connect_sqlserver_windowsauth('bidatacentre',db)
# Table Details
tab_det_df = fetch_table_details(engine)
tab_dates = fetch_table_dates_and_rowcount(engine)
# View Details
vcd = fetch_view_column_details(engine)
view_def = fetch_view_definitions(engine)

# Get all DB Dependencies
deps_df = fetch_all_db_dependencies(engine)


In [118]:
vcd['view_name'].unique()

array(['AD_PRDProject_List', 'AD_User_Details', 'AX_AP_SCVendor',
       'AX_Attachment_JournalHeader', 'AX_Attachment_JournalLines',
       'AX_BOL_Details', 'AX_Budget_Trans', 'AX_COMReceivedVolumes',
       'AX_COMReceivedVolumesByWell', 'AX_Contact_Master',
       'AX_Currency_Conversion', 'AX_Customer_Address',
       'AX_Customer_Contacts', 'AX_Customer_Master',
       'AX_Customer_Master_Last30', 'AX_Customer_Trans',
       'AX_Default_Dimensions', 'AX_Dimension_BusinessUnit',
       'AX_Dimension_Division', 'AX_Dimension_LegalEntity',
       'AX_Dimension_Location', 'AX_Facility_UserAccess',
       'AX_FixedAsset_Master', 'AX_FixedAsset_TransDetails',
       'AX_GL_Master', 'AX_GL_TotalSummary', 'AX_GL_TransDetails',
       'AX_Inventory_Location', 'AX_Item_CPOnHandTEST',
       'AX_Item_PCConvert', 'AX_Item_PRDGLVolumeMap',
       'AX_Item_ProdChemBOL', 'AX_Item_ReleasedProductMaster',
       'AX_Item_SCItemList', 'AX_Item_UnitIDMapping',
       'AX_Journal_LedgerTrans', 'AX_J

In [145]:
#for view_name in list(vcd['view_name']):
view_name = vcd['view_name'][0]
vcd_filter = vcd['view_name'] == view_name
view_def_filter = view_def['view_name'] == view_name
deps_parent_filter = deps_df['referencing_object_name'] == view_name
deps_child_filter = deps_df['referenced_entity_name'] == view_name

view_name

'AD_PRDProject_List'

In [146]:
vcd[ vcd_filter ]

,schema_name,view_name,column_name,data_type,data_type_ext,nullable,comments
0,dbo,AD_PRDProject_List,Department,varchar,varchar(100),Y,None
1,dbo,AD_PRDProject_List,Email,varchar,varchar(100),N,None
2,dbo,AD_PRDProject_List,FullName,varchar,varchar(511),N,None
3,dbo,AD_PRDProject_List,GroupEmail,varchar,varchar(100),Y,None
4,dbo,AD_PRDProject_List,GroupName,varchar,varchar(100),N,None
5,dbo,AD_PRDProject_List,Title,varchar,varchar(100),Y,None


In [147]:
view_def[ view_def_filter ]

,schema_name,view_name,created,last_modified,definition,comments
0,dbo,AD_PRDProject_List,2018-12-03 13:05:42.303,2018-12-03 13:05:42.303,\r\nCREATE VIEW dbo.AD_PRDProject_List\r\nAS\r...,None


In [148]:
view_def[ view_def_filter ]['definition'][0]
reformatted = sqlparse.format(view_def[ view_def_filter ]['definition'][0], reindent=True, keyword_case='upper')
print(reformatted)


CREATE VIEW dbo.AD_PRDProject_List AS
SELECT ed.GroupName ,
       ed.GroupEmail ,
       ISNULL(nullif(ec.FirstName, '') + ' ', '') + isnull(ec.LastName, '') AS FullName ,
       ed.MemberEmail AS Email ,
       ec.Title ,
       ec.Dept AS Department
FROM dbo.Exchange_DistList ed (nolock)
LEFT JOIN dbo.Exchange_GAL_Contacts ec (nolock) ON ec.Email = ed.MemberEmail
WHERE ed.GroupName = 'PRD Projects' ;


In [149]:
deps_df.loc[ deps_parent_filter ]

,DATABASE,referencing_schema_name,referencing_object_name,referencing_type_desc,referenced_server_name,referenced_db_name,referenced_schema_name,referenced_entity_name
0,BI,dbo,AD_PRDProject_List,VIEW,None,BI,dbo,Exchange_DistList
1,BI,dbo,AD_PRDProject_List,VIEW,None,BI,dbo,Exchange_GAL_Contacts


In [150]:
deps_df.loc[ deps_child_filter ]

,DATABASE,referencing_schema_name,referencing_object_name,referencing_type_desc,referenced_server_name,referenced_db_name,referenced_schema_name,referenced_entity_name


In [154]:
view_schema = vcd.loc[vcd_filter]['schema_name'][0]
view_schema

'dbo'

In [155]:
vcd.loc['view_name'] == view_name

KeyError: 'view_name'

In [156]:
view_name

'AD_PRDProject_List'

In [157]:
vcd

,schema_name,view_name,column_name,data_type,data_type_ext,nullable,comments
0,dbo,AD_PRDProject_List,Department,varchar,varchar(100),Y,None
1,dbo,AD_PRDProject_List,Email,varchar,varchar(100),N,None
2,dbo,AD_PRDProject_List,FullName,varchar,varchar(511),N,None
3,dbo,AD_PRDProject_List,GroupEmail,varchar,varchar(100),Y,None
4,dbo,AD_PRDProject_List,GroupName,varchar,varchar(100),N,None
...,...,...,...,...,...,...,...
5643,stage,Avetta_Person_Role_Status_Report_TRANSFORM,Role,nvarchar,nvarchar(4000),Y,None
5644,stage,Avetta_Person_Role_Status_Report_TRANSFORM,Role_Expiry,date,date,Y,None
5645,stage,Avetta_Person_Role_Status_Report_TRANSFORM,Role_ID,int,int,Y,None
5646,stage,Avetta_Person_Role_Status_Report_TRANSFORM,Status,nvarchar,nvarchar(50),Y,None


In [173]:
vcd_filter = vcd['view_name'] == 'AD_User_Details'
view_schema = vcd.loc[vcd_filter]


'dbo'

In [174]:
view_def

,schema_name,view_name,created,last_modified,definition,comments
0,dbo,AD_PRDProject_List,2018-12-03 13:05:42.303,2018-12-03 13:05:42.303,\r\nCREATE VIEW dbo.AD_PRDProject_List\r\nAS\r...,None
1,dbo,AD_User_Details,2017-03-09 17:35:53.943,2019-01-30 11:16:17.153,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nCREATE VIE...,None
2,dbo,AX_AP_SCVendor,2015-10-20 11:29:56.300,2015-10-26 09:57:14.750,\r\n\r\n\r\n\r\nCREATE VIEW [dbo].[AX_AP_SCVen...,None
3,dbo,AX_Attachment_JournalHeader,2017-02-28 13:11:43.113,2017-02-28 13:11:43.113,\r\n\r\n\r\n\r\n\r\nCREATE VIEW [dbo].[AX_Atta...,None
4,dbo,AX_Attachment_JournalLines,2017-02-28 10:40:56.417,2017-02-28 13:23:05.813,\r\n\r\n\r\n\r\n\r\n\r\nCREATE VIEW [dbo].[AX_...,None
...,...,...,...,...,...,...
209,dbo,zSSRS_OI_File_POInvoice,2017-03-27 15:22:15.720,2017-03-28 14:12:52.620,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nCREATE...,None
210,dbo,zSSRS_SOSU_ProjectGLTrans,2017-12-28 08:10:38.410,2018-01-05 10:01:21.580,\r\n\r\n\r\n\r\n\r\n\r\n\r\n/*****************...,None
211,dbo,zUltipro_InternalPostingstoCSV,2018-02-12 09:19:15.763,2019-02-14 11:43:26.717,\r\n\r\n\r\nCREATE VIEW [dbo].[zUltipro_Intern...,None
212,dbo,zUpload_LBZ_ItemMaster,2017-05-23 19:26:20.800,2018-11-13 08:48:05.333,/*\r\nCreated: \r\n\tUnknown\r\n\r\nModified:\...,None


In [175]:
view_def.loc[view_def_filter]['definition]

SyntaxError: EOL while scanning string literal (66175917.py, line 1)

# GENERATE .md files for use in obsidian or wiki

In [82]:
# Generate base directories + include directories
DataCatalog_Path = Path("C:/obsidian/datacatalog/bidatacentre")
Include_Path = DataCatalog_Path / "include"

for db in list(db_df['name']):
    engine = connect_sqlserver_windowsauth('bidatacentre',db)
    # Table Details
    tab_det_df = fetch_table_details(engine)
    tab_dates = fetch_table_dates_and_rowcount(engine)
    # View Details

    # Get all DB Dependencies
    deps_df = fetch_all_db_dependencies(engine)

    for db_schema in list(tab_det_df['schema_name'].unique()):
        (Include_Path / db / db_schema).mkdir(parents=True, exist_ok=True)
        (DataCatalog_Path / db / db_schema).mkdir(parents=True, exist_ok=True)
        # drill down to individual schema / table and generate files.
        for table in list((tab_det_df.loc[tab_det_df['schema_name'] == db_schema])['table_name'].unique()):
            # Just the column details for this table
            tab_df = tab_det_df.loc[ (tab_det_df['schema_name'] == db_schema) & (tab_det_df['table_name'] == table) ]
            tab_df = tab_df[['column_name',
                             'data_type_ext',
                             'nullable',
                             'default_value',
                             'primary_key',
                             'foreign_key',
                             'unique_key',
                             'comments']]
            tab_df.rename(columns={'column_name': 'Column',
                     'data_type_ext': 'DataType',
                     'nullable': 'Nullable',
                     'default_value': 'Default',
                     'primary_key': 'IS_PK',
                     'foreign_key': 'IS_FK',
                     'unique_key': 'Unique',
                     'comments': 'Comments'}, inplace=True)

            tabfile = table + ".md"
            row = tab_dates.loc[(tab_dates['schema_name'] == db_schema) & (tab_dates['table_name'] == table)]
            with open ((DataCatalog_Path / db / db_schema / tabfile ), mode='w') as file:
                file.write("##### SUMMARY :\n\n")
                file.write(f"DB: **{db}**  SCHEMA: **{db_schema}**  OBJECT: **{table}** \n")
                file.write(f"Created: **{row.iloc[0]['created']}**  Modified: **{row.iloc[0]['last_modified']}**\n")
                file.write(f"Rows: **{row.iloc[0]['num_rows']}**\nComments: **{row.iloc[0]['comments']}**\n")
                file.write("\n\n")

                file.write("##### DEPENDENCIES :\n\n")
                file.write("Parents:\n")
                parents = get_parents(deps_df, db, db_schema, object)
                if len(parents) >= 1:
                    file.write(markdown_table(parents))
                else:
                    file.write("**None**")
                file.write("\n\n")

                file.write("Children:\n")
                children = get_children(deps_df, db, db_schema, object)
                if len(children) >= 1:
                    print(children)
                    file.write(markdown_table(children))
                else:
                    file.write("**None**")
                file.write("\n\n")

                file.write("##### COLUMN DEFINITIONS:\n\n")
                file.write(markdown_table(tab_df))
                file.write("\n\n")
                file.write("##### EXPLORATORY DATA ANALYSIS:\n\n")
                # Use actual data for tables with less than 1000 rows
                if row.iloc[0]['num_rows'] <= 1000:
                    rows = fetch_all_rows(engine, db_schema, table)
                    # Using pandas_profiling report for simple EDA
                    profile = ProfileReport(rows, title=table)
                    tabfile = f"{table}_profile.html"
                    filename = (Include_Path / db / db_schema / tabfile )
                    profile.to_file(filename)
                    file.write(f'<iframe src="{filename}" height=300 width=1000>\n\n')
                    file.write(f"Open in browser: ![[{tabfile}]]\n")
                    file.write("\n\n")
                else:
                    # Methods to sample a random percentage of a table in MS SQL Server
                    # https://docs.microsoft.com/en-us/previous-versions/sql/sql-server-2008-r2/ms189108(v=sql.105)?redirectedfrom=MSDN
                    #rows = fetch_sample_rows(engine, db_schema, table, sample_pct=10)

                    file.write("\n\n")

                file.write("##### USER NOTES:\n\n")
                file.write(f"![[include/{db}/{db_schema}/{table}_user_notes.md]]\n\n")



NameError: name 'deps' is not defined